In [ ]:
import re
from dataclasses import dataclass
from typing import List, Tuple, Optional

@dataclass
class DetectionResult:
    is_injection: bool
    confidence: float
    matched_rules: List[str]
    severity: str

def create_injection_detector(
    custom_patterns: Optional[List[Tuple[str, float]]] = None
):
    base_patterns = [
        (r"(?i)(игнорируй|забудь).*(предыдущ|вышесказанн)", 0.8),
        (r"(?i)(ignore|disregard).*(previous|above)", 0.8),
        (r"(?i)системн.*сообщен", 0.9),
        (r"(?i)system.*prompt", 0.9),
        (r"(?i)(execute|sudo|rm\s+-rf)", 1.0),
        (r"(?i)теперь\s+ты", 0.7),
        (r"(?i)act\s+as", 0.7),
        (r"(?i)<script.*>", 1.0),
        (r"(?i)(SELECT|INSERT|UPDATE|DELETE|DROP).*FROM", 0.9),
        (r"(?i)перезаписать.*контекст", 0.6),
        (r"(?i)инъекция", 0.8),
        (r"(?i)injection", 0.8)
    ]
    
    all_patterns = base_patterns.copy()
    if custom_patterns:
        all_patterns.extend(custom_patterns)
    
    def detect_prompt_injection(
        text: str, 
        threshold: float = 0.7
    ) -> DetectionResult:
        matched_rules = []
        total_score = 0.0
        
        for pattern, weight in all_patterns:
            if re.search(pattern, text, re.IGNORECASE | re.DOTALL):
                matched_rules.append(pattern)
                total_score += weight
        
        max_possible_score = sum(w for _, w in all_patterns)
        confidence = total_score / max_possible_score if max_possible_score > 0 else 0
        
        # severity
        if confidence >= 0.9:
            severity = "CRITICAL"
        elif confidence >= 0.7:
            severity = "HIGH"
        elif confidence >= 0.4:
            severity = "MEDIUM"
        else:
            severity = "LOW"
        
        return DetectionResult(
            is_injection=confidence >= threshold,
            confidence=confidence,
            matched_rules=matched_rules,
            severity=severity
        )
    
    return detect_prompt_injection